In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from gpu_spatial_graph_pipeline.utils import adata2data
from gpu_spatial_graph_pipeline.data.node_datamodule import NodeDataModule
from gpu_spatial_graph_pipeline.models.graph_embedding import GraphEmbedding
from gpu_spatial_graph_pipeline.models.non_linear_ncem import NonLinearNCEM
from gpu_spatial_graph_pipeline.models.linear_ncem import LinearNCEM

In [3]:
#Mibitof
# adata = sq.datasets.mibitof()
# #feature_name=adata.obs.keys()[0] #Use for IMC dataset
# feature_name='Cluster'
# num_features=len(set(adata.obs[feature_name]))
# encoder_hidden_dims=30
# decoder_hidden_dims=30
# num_vars=adata.X.shape[1]

#IMC


adata = sq.datasets.imc()
feature_name=adata.obs.keys()[0] #Use for IMC dataset

def adata2data_imc(adata: AnnData):
    return adata2data(adata,feature_name)

num_features=len(set(adata.obs[feature_name]))
num_vars=adata.X.shape[1]


In [4]:
dm = NodeDataModule(adata=adata,adata2data_fn=adata2data_imc, num_workers = 8, batch_size=100)

In [5]:
dm.setup()

In [7]:
for batch in dm.train_dataloader():
    print(batch)

Data(num_nodes=47, edge_index=[2, 0], x=[47, 11], y=[47, 34])
Data(num_nodes=56, edge_index=[2, 0], x=[56, 11], y=[56, 34])
Data(num_nodes=45, edge_index=[2, 2], x=[45, 11], y=[45, 34])
Data(num_nodes=44, edge_index=[2, 0], x=[44, 11], y=[44, 34])
Data(num_nodes=39, edge_index=[2, 4], x=[39, 11], y=[39, 34])
Data(num_nodes=52, edge_index=[2, 6], x=[52, 11], y=[52, 34])
Data(num_nodes=53, edge_index=[2, 4], x=[53, 11], y=[53, 34])
Data(num_nodes=40, edge_index=[2, 2], x=[40, 11], y=[40, 34])
Data(num_nodes=53, edge_index=[2, 6], x=[53, 11], y=[53, 34])
Data(num_nodes=45, edge_index=[2, 1], x=[45, 11], y=[45, 34])
Data(num_nodes=44, edge_index=[2, 0], x=[44, 11], y=[44, 34])
Data(num_nodes=40, edge_index=[2, 2], x=[40, 11], y=[40, 34])
Data(num_nodes=39, edge_index=[2, 4], x=[39, 11], y=[39, 34])
Data(num_nodes=40, edge_index=[2, 0], x=[40, 11], y=[40, 34])
Data(num_nodes=55, edge_index=[2, 0], x=[55, 11], y=[55, 34])
Data(num_nodes=42, edge_index=[2, 5], x=[42, 11], y=[42, 34])
Data(num

In [8]:
model = LinearNCEM(in_channels=num_features,out_channels=num_vars, lr=0.0001,weight_decay=0.000001)

In [9]:
gpu=False
if gpu:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=100,log_every_n_steps=1)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=100,log_every_n_steps=1)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sel/anaconda3/envs/ncem/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [10]:
trainer.fit(model,datamodule=dm)


  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 408   
1 | model_mu    | LinearSpatial   | 408   
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
816       Trainable params
0         Non-trainable params
816       Total params
0.003     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/sel/anaconda3/envs/ncem/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/sel/anaconda3/envs/ncem/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
